In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def viterbi(obs_space, state_space, init_probs, obs_seq, trans_probs, obs_probs):
    T = len(obs_seq)
    K = len(state_space)
    t1 = np.zeros(shape=(K, T)) #t1[i, j] probability of likeliest path so far (observing y_j) ending in state x_i
    t2 = np.zeros(shape=(K, T), dtype=int) #saves the previous state that led to t1[i, j] becoming maximal, allowing backtracing
    y = np.array([obs_space.index(obs) for obs in obs_seq]) #save observation sequence as integer array
    states = np.array([i for i in range(K)]) #associate each state with an integer index
    p = np.asarray(init_probs)
    A = np.asarray(trans_probs)
    B = np.asarray(obs_probs)
    
    t1[:, 0] = p + B[:,y[0]]
    for j in range(1,T):
        for i in range(K):
            t1[i, j] = B[i, y[j]] + np.max(t1[:, j-1]+A[:, i])
            t2[i, j] = np.argmax(t1[:, j-1]+A[:, i])
            
    optimal_sequence = np.zeros(T, dtype=int)
    z = np.argmax(t1[:, -1])
    optimal_sequence[-1] = states[z]
    for j in range(T-1,1, -1):
        z = t2[z, j]
        optimal_sequence[j-1] = states[z]
    optimal_state_sequence = [state_space[state_index] for state_index in optimal_sequence]
    return t1, optimal_state_sequence    

In [ ]:
state_space = ["L", "H"]
obs_space = ["A", "C", "G", "T"]
init_probs = [-1, -1]
trans_probs = [[-0.737, -1.322],
               [-1.000, -1.000]]
obs_probs = [[-1.737, -2.322, -2.322, -1.737],
             [-2.322, -1.737, -1.737, -2.322]]
obs_sequence = ["G","G","C","A","C","T","G","A","A"]

In [ ]:
table, optimal_sequence = viterbi(obs_space, state_space, init_probs, obs_sequence, trans_probs, obs_probs)

In [ ]:
#table of log probabilities of most likely paths ending with hidden state H or L for given observed sequence 
pd.DataFrame(data=table[:][::-1], columns=obs_sequence, index=state_space[::-1])

In [ ]:
#sequence of most likely hidden states given the observed sequence
pd.DataFrame(data=[optimal_sequence], columns=obs_sequence, index=["most likely sequence"])